In [34]:
# MIT License
# Copyright (c) 2025 Mahtab Syed
# https://www.linkedin.com/in/mahtabsyed/
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so.

# 16 - Resource-Aware Optimization

- Prompt Classification : Determines what the user's input is (uses gpt-4o model with a strict JSON-only response format)
    - "simple" – answerable directly
    - "reasoning" – requires logic or multi-step thought
    - "internet_search" – requires up-to-date external information
- If "simple" uses - "gpt-4o-mini"
- If "reasoning" uses - "o4-mini"
- If "internet search" - call Google Search API and then passes response to "gpt-4o" 


- Prerequisites
    - OpenAI Key with access to "gpt-4o", "gpt-4o-mini", "o4-mini"
    - Google Programmable Search Engine (CSE):https://programmablesearchengine.google.com/
    - Note to enable Custom Search API in Google
    - Google API Key and CSE ID from:https://console.cloud.google.com/apis
    - Save these to a .env file


In [35]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI


# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_CUSTOM_SEARCH_API_KEY = os.getenv("GOOGLE_CUSTOM_SEARCH_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

if not OPENAI_API_KEY or not GOOGLE_CUSTOM_SEARCH_API_KEY or not GOOGLE_CSE_ID:
    raise ValueError(
        "Please set OPENAI_API_KEY, GOOGLE_CUSTOM_SEARCH_API_KEY, and GOOGLE_CSE_ID in your .env file."
    )

client = OpenAI(api_key=OPENAI_API_KEY)


# --- Step 1: Classify the Prompt ---
def classify_prompt(prompt: str) -> dict:
    system_message = {
        "role": "system",
        "content": (
            "You are a classifier that analyzes user prompts and returns one of three categories ONLY:\n\n"
            "- simple\n"
            "- reasoning\n"
            "- internet_search\n\n"
            "Rules:\n"
            "- Use 'simple' for direct factual questions that need no reasoning or current events.\n"
            "- Use 'reasoning' for logic, math, or multi-step inference questions.\n"
            "- Use 'internet_search' if the prompt refers to current events, recent data, or things not in your training data.\n\n"
            "Respond ONLY with JSON like:\n"
            '{ "classification": "simple" }'
        ),
    }

    user_message = {"role": "user", "content": prompt}

    response = client.chat.completions.create(
        model="gpt-4o", messages=[system_message, user_message], temperature=1
    )

    reply = response.choices[0].message.content
    return json.loads(reply)


# --- Step 2: Google Search ---
def google_search(query: str, num_results=1) -> list:
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_CUSTOM_SEARCH_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "q": query,
        "num": num_results,
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        results = response.json()

        if "items" in results and results["items"]:
            return [
                {
                    "title": item.get("title"),
                    "snippet": item.get("snippet"),
                    "link": item.get("link"),
                }
                for item in results["items"]
            ]
        else:
            return []
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}


# --- Step 3: Generate Response ---
def generate_response(prompt: str, classification: str, search_results=None) -> str:
    if classification == "simple":
        model = "gpt-4o-mini"
        full_prompt = prompt
    elif classification == "reasoning":
        model = "o4-mini"
        full_prompt = prompt
    elif classification == "internet_search":
        model = "gpt-4o"
        # Convert each search result dict to a readable string
        if search_results:
            search_context = "\n".join(
                [
                    f"Title: {item.get('title')}\nSnippet: {item.get('snippet')}\nLink: {item.get('link')}"
                    for item in search_results
                ]
            )
        else:
            search_context = "No search results found."
        full_prompt = f"""Use the following web results to answer the user query:

{search_context}

Query: {prompt}"""

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": full_prompt}],
        temperature=1,
    )

    return response.choices[0].message.content, model


# --- Step 4: Combined Router ---
def handle_prompt(prompt: str) -> dict:
    classification_result = classify_prompt(prompt)
    # Remove or comment out the next line to avoid duplicate printing
    # print("\n🔍 Classification Result:", classification_result)
    classification = classification_result["classification"]

    search_results = None
    if classification == "internet_search":
        search_results = google_search(prompt)
        # print("\n🔍 Search Results:", search_results)

    answer, model = generate_response(prompt, classification, search_results)
    return {"classification": classification, "response": answer, "model": model}

In [36]:
test_prompt = "What is the capital of Australia?"
# test_prompt = "Explain the impact of quantum computing on cryptography."
# test_prompt = "When does the Australian Open 2026 start, give me full date?"

result = handle_prompt(test_prompt)
print("🔍 Classification:", result["classification"])
print("🧠 Model Used:", result["model"])
print("🧠 Response:\n", result["response"])

🔍 Classification: simple
🧠 Model Used: gpt-4o-mini
🧠 Response:
 The capital of Australia is Canberra.


In [37]:
# test_prompt = "What is the capital of Australia?"
test_prompt = "When does the Australian Open 2026 start, give me full date with year?"
# test_prompt = "Explain the impact of quantum computing on cryptography."

result = handle_prompt(test_prompt)
print("🔍 Classification:", result["classification"])
print("🧠 Model Used:", result["model"])
print("🧠 Response:\n", result["response"])

🔍 Classification: internet_search
🧠 Model Used: gpt-4o
🧠 Response:
 The Australian Open 2026 starts on January 19, 2026.


In [38]:
# test_prompt = "What is the capital of Australia?"
# test_prompt = "When does the Australian Open 2026 start, give me full date?"
test_prompt = "Explain the impact of quantum computing on cryptography."

result = handle_prompt(test_prompt)
print("🔍 Classification:", result["classification"])
print("🧠 Model Used:", result["model"])
print("🧠 Response:\n", result["response"])

🔍 Classification: reasoning
🧠 Model Used: o4-mini
🧠 Response:
 Quantum computing poses both threats and opportunities for modern cryptography. Here’s an overview of the key impacts:

1. Threats to Asymmetric (Public-Key) Cryptography  
  • Shor’s Algorithm  
    – Efficiently factors large integers and computes discrete logarithms in polynomial time.  
    – Breaks RSA, DH (Diffie–Hellman), DSA and ECC (elliptic-curve) schemes once a sufficiently large, error-corrected quantum computer exists.  
  • Consequences  
    – Any data encrypted today with vulnerable public-key systems could eventually be decrypted (“harvest now, decrypt later”).  
    – Digital signatures based on these schemes become forgeable.

2. Impact on Symmetric Cryptography  
  • Grover’s Algorithm  
    – Provides a quadratic speedup for unstructured search (e.g., brute-forcing symmetric keys).  
    – Reduces effective key length by half: an n-bit key has security roughly 2^(n/2) instead of 2^n.  
  • Mitigation  
